## Transmission Profile Processing

This notebook processes the data collected in the measurement notebook to produce Signal-to-Noise plots and the final transmission profile of the sapphire coating sample.

Sam Condon \
Caltech \
05/18/2021

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
## Convert existing sample_log.txt file to a .cfg for easy read in to a pandas dictionary

sample_log_txt = open("sample_log.txt")


In [22]:
sample_log_txt.tell()
sample_log_txt.read()

''

In [23]:
sample_log_txt.close()